In [2]:
import pandas as pd
import numpy as np
from gcmap import GCMapper, Gradient
from PIL import Image, ImageFilter

ModuleNotFoundError: No module named 'gcmap'

In [ ]:
ROUTE_COLS = ('airline_name', 'airline_id', 'source_code', 'source_id', 'dest_code', 'dest_id', 'codeshare', 'stops', 'equiptment')
AIRPORT_COLS = ('airport_id', 'airport_name', 'city', 'country', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'timezone', 'dst', 'tz database', 'type', 'source')

routes = pd.read_csv('data/routes.dat', header=None, names=ROUTE_COLS, na_values=['\\N'])
airports = pd.read_csv('data/airports.dat', header=None, names=AIRPORT_COLS)

In [ ]:
routes.drop(['codeshare', 'stops', 'equiptment'], axis=1)

In [ ]:
airports.drop(['icao', 'timezone', 'dst', 'tz database', 'source'], axis=1)

In [ ]:
airport_pairs = routes.groupby(('source_id', 'dest_id')).size()
airport_pairs = airport_pairs.reset_index()
airport_pairs.columns = ('source_id', 'dest_id', 'cnt')

In [ ]:
airport_pairs

In [ ]:
airport_pairs = airport_pairs.merge(airports, left_on='source_id', right_on='airport_id') \
                            .merge(airports, left_on='dest_id', right_on='airport_id', suffixes=('_source', '_dest'))

In [ ]:
g = Gradient([(   0,   204,   255,   131),
              ( 0.4,    51,   255,    51),
              (   1,     0,     0,     0)])


gcm = GCMapper(cols=g, bgcol=(255,255,255))
gcm.set_data(airport_pairs.longitude_source, airport_pairs.latitude_source, airport_pairs.longitude_dest, airport_pairs.latitude_dest, airport_pairs.cnt)
img = gcm.draw()
img.save('test.png')

original = Image.open("test.png")
original.show()

# Creating Inter-country Routes (in .csv)

In [ ]:
airport_pairs

In [ ]:
country_routes = airport_pairs.groupby(('country_source', 'country_dest'))
country_routes = country_routes.aggregate(np.sum)

In [ ]:
country_routes

In [ ]:
country_routes.loc['China',:]

In [ ]:
country_routes = country_routes.reset_index()
ext_country_routes = country_routes[country_routes['country_source'] != country_routes['country_dest']]
ext_country_routes

In [ ]:
ext_country_routes.to_csv (r'/home/jan/Code/MasterThesis/data/country_flight_country.csv', index = False, header=True)